# Martech - Query Data

## Initialise Connection

In [ ]:
from datetime import datetime
from astra_db import AstraSession
from state import OffsetState as os

astra = AstraSession()

In [ ]:
# Prepare Statements

# comm_by_activity_day
get_acty_cap_stmt = astra.session.prepare("""SELECT comm_time_bucket, comm_offset_time, comm_offset_count 
                                          FROM comm_by_activity_day WHERE 
                                          activity_name = ? AND comm_date_bucket = ? AND comm_time_bucket > ?""")

update_acty_cap_stmt = astra.session.prepare("""UPDATE comm_by_activity_day
                                             SET comm_offset_time = ?, comm_offset_count = ? 
                                             WHERE activity_name = ? AND comm_date_bucket = ?""")


## Define Entities

In [ ]:
# Define Entities

# customer_id
customer_id_array = [
    '0b6ec30f-8b1a-406f-82f6-89222a017449',
    '8435a9b9-9e88-495c-a297-dfd25d44428b',
    '8e12d570-e47d-43b9-9435-b2020899860b',
    '9e53cd14-0316-46ff-b913-30228fcc5b73',
    'a6e198e0-674d-4a27-ba01-595a78fda2ef',
    'af9e65bd-4474-40c9-8d16-7b82035d3e0f',
    'b191cef3-4624-4e88-9a93-73de506e4be7',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
    'cf3a5d68-e66f-4ee7-a2d1-2666c5366540',
    'f68e8bc9-86b7-4c8e-ba74-87d31d27a531',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
]

# channel
channel_array = ['email', 'sms', 'push', 'in-app', 'direct']

# category_group
category_group_array = ['ctgy_grp_1', 'ctgy_grp_2', 'ctgy_grp_3', 'ctgy_grp_4', 'ctgy_grp_5']

# category
category_array = ['ctgy_1', 'ctgy_2', 'ctgy_3', 'ctgy_4', 'ctgy_5']

# activity_name
activity_name_array = ['credit_card', 'mortgage', 'business', 'retail', 'investment']

## Query Data

In [ ]:
activity_name = 'retail'

### ACTYCAP

#### Query

In [ ]:
# Communications by Activity Day
"""
SELECT comm_time_bucket, comm_offset_time, comm_offset_count FROM comm_by_activity_day 
WHERE activity_name = ? AND comm_date_bucket = ? AND comm_time_bucket > ? """

#
# To get the total count, we add comm_offset_count to the number of rows returned


# Set the current day and time buckets
date = datetime.now()
current_date_bucket = int(date.strftime('%Y%m%d')) # bucket by day
current_time_bucket = int(date.strftime('%H%M%S%f')[:8]) # format time as HHMMSSssss

# Read stored state values for activity from JSON file named state.json
(comm_offset_time, comm_offset_count) = os.get_state('comm_by_activity_day')

print(f"State Offset Time: { comm_offset_time}  State Offset Count: {comm_offset_count}")


rows = astra.session.execute(get_acty_cap_stmt, [activity_name, current_date_bucket, comm_offset_time ])
# test if query returned any rows
if not rows:
    print("No rows returned")
    row_count = 0
    total_comm_count = comm_offset_count
else:
    comm_offset_time = rows[0].comm_offset_time or 0
    comm_offset_count = rows[0].comm_offset_count or 0
    comm_time_bucket = rows[0].comm_time_bucket or 0
    row_count = sum(1 for _ in rows)  # Count rows
    total_comm_count = comm_offset_count + row_count

print(f"Total Count: {total_comm_count},  Row Count: {row_count}")
print(f"Comm Time Bucket: {current_time_bucket}  Offset Time: { comm_offset_time}  Offset Count: {comm_offset_count}")

State Offset Time: 18202046  State Offset Count: 4
State Offset Time: 18202046  State Offset Count: 4
No rows returned
Total Count: 4,  Row Count: 0
Comm Time Bucket: 18205219  Offset Time: 18202046  Offset Count: 4


#### Update

In [ ]:
# Update - Communications by Activity Day
# SET comm_offset_time = ?, comm_offset_count = ? WHERE activity_name = ? AND comm_date_bucket = ?
#
# comm_offset_time will be the last comm_time value read
# comm_offset_count will be the number of comm_time values counted up to that time.

rows = astra.session.execute(update_acty_cap_stmt, [comm_time_bucket, total_comm_count, activity_name, current_date_bucket])

# Store State for next run
os.set_state('comm_by_activity_day', comm_time_bucket, total_comm_count)

## Cleanup

In [11]:
astra.shutdown()